In [32]:
import threading as td # Python的多執行緒函式庫
import tkinter as tk # Python官方欽定的GUI函式庫
import cv2    # 全名是OpenCV，主要負責影像處理
from PIL import Image, ImageTk, Image # tkinter可以接受的圖片
import numpy as np # C 和 Fortran撰寫的超強大矩陣計算函式庫
import face_recognition # 人臉辨識的函式庫
import random # 隨機
import time   # Python
import dlib   # C++所撰寫的工具庫，內容包含 機器學習的演算法
import pickle # 可以將python變數.二進資料進行雙向轉換二進制的數據格式
import json
from tkinter import filedialog, messagebox,SEPARATOR, ttk # tkinter的ㄟGUI

# COMPARE_THRESHOLD 人臉比對的門檻值，數字越小會越嚴格比對但太小反而認不出
# (反之，數字如果設置太大，越容易有認錯人的情形發生！)
COMPARE_THRESHOLD = 0.4

# 方塊顏色
RECTANGLE_COLOR = (0,165,255) 

# 將cv2讀取的圖片轉成ImageTk並回傳
def cv2Tk_image(cv_img):
    cv2image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    return imgtk

In [40]:

class ImageAnalysisFrame:    
    def read_image(self):
        image_file_names = filedialog.askopenfilenames(filetypes=(('JPG', ".jpg"), ('JPEG', ".jpeg"), ('PNG', '.png')))
        if len(image_file_names) == 0:
            return
        
        self.image_names= image_file_names
        
        for image_file_name in image_file_names:
            print('file_path :', image_file_name)
            self.message_label.config(text=image_file_name, width=len(image_file_name))
            self.analysis_faces_btn.pack()            
#         else:
#             # 沒有成功讀取圖片，就不能執行 人臉辨識
#             self.analysis_faces_btn.forget()
#             self.show_image_label.forget()
#             print('讀取失敗')
    
    def face_detection(self, cvImg, options=None):
        """
        option : {"locations": 1, "graphic":1}
        
        """
        if options is None:
            options = {"locations": 1, "graphic":1}
        
        data = {}

        face_locations = face_recognition.face_locations(cvImg)
        data['locations'] = face_locations
            
        if options.get('graphic', None):
            face_encodings = face_recognition.face_encodings(cvImg, face_locations)
            data['graphic'] = cvImg.copy()
            
            # 比對到的人名
            face_names = []
            known_encodings = self.train_data['face_encodings']
            
            # 已知的人名集合(不重複)
            face_names_set = set(self.train_data['face_names'])
            
            # 已知的人名對應出現次數的字典
            face_names_times_dict = {}
            
            
                
            # 找出所有臉中是否有比對到臉的，有的話就添加(投票機制)
            for face_encoding in face_encodings:
                match = face_recognition.compare_faces(known_encodings , face_encoding, COMPARE_THRESHOLD)
                name = "Unknown"
                if len(known_encodings) > 0:
                    print('match:', match)

                    # 初始化 人名對次數的字典
                    for name in face_names_set:
                        face_names_times_dict[name] = 0

                    # 第一個比對成功到的臉的名字，不管之後有沒有類似的(投票機制)
                    for inx, value in enumerate(match):
                        if value:
                            name = self.train_data['face_names'][inx]
                            face_names_times_dict[name] += 1
                            
                    # 如果字典最大值等於0，代表說沒有一張臉比對成功
                    print('max(face_names_times_dict) = ', max(face_names_times_dict))
                    if [value for (inx, value) in face_names_times_dict.items() if value == max(face_names_times_dict.values())][0] <= 0:
                        face_names.append("Unknown")
                        continue
                        
                    # 找出字典中值最大的key（此邊的Key為 樣本名稱）
                    face_names.append(max(face_names_times_dict, key=face_names_times_dict.get))
                        
                else: # train_data什麼資料都沒有，直接判定unknow
                    face_names.append(name)
            
            # 將face_names(人名)也放進data中，索引跟data['locations']是相對的。
            data['face_names'] = face_names
            
            # 將捕捉到的臉用藍色框框畫在圖片上
            for inx, (top, right, bottom, left) in enumerate(face_locations):
                cv2.rectangle(data['graphic'], (left, top), (right, bottom), (0, 0, 255), 2)
                
                # Draw a label with a name below the face
                cv2.rectangle(data['graphic'], (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(data['graphic'], face_names[inx], (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        return data
    
    def face_detect_show_image(self,):
        # 成功配對的圖片資料(包含人名, 位置和 繪製的圖片)
        self.success_match_dict = {'image_path':[], 'face_names': [], 'graphic': [], 'locations': []}
        self.fail_match_dict = {'image_path':[], 'face_names': [], 'graphic': [], 'locations': []}
        
        # 清除掉左邊成功&失敗清單的所有內容！
        self.success_lb.delete(0, tk.END)
        self.fail_lb.delete(0, tk.END)
        
        for image_name in self.image_names:
            cvImage = cv2.imread(image_name)
            detection_result = self.face_detection(cvImage)
            
            # 沒有比對成功的一張圖，將該圖的data放入self.fail_match_dict
            if len(detection_result['face_names']) == 0:
                self.fail_match_dict['image_path'].append(image_name)
                self.fail_match_dict['face_names'].append(detection_result['face_names']) 
                self.fail_match_dict['graphic'].append(detection_result['graphic']) 
                self.fail_match_dict['locations'].append(detection_result['locations'])
                
                # 失敗的路徑名 添加在 左手邊的失敗列表
                self.fail_lb.insert('end', image_name)
            else:
                # 比對成功的一張圖，將該圖的data放入self.success_match_dict
                
                self.success_match_dict['image_path'].append(image_name)
                self.success_match_dict['face_names'].append(detection_result['face_names']) 
                self.success_match_dict['graphic'].append(detection_result['graphic']) 
                self.success_match_dict['locations'].append(detection_result['locations']) 
                # 成功的路徑名 添加在 左手邊的成功列表
                self.success_lb.insert('end', image_name)
                
        # 計算成功張數 和 失敗張數
        sucs_count = len(self.success_match_dict['image_path'])
        fail_count = len(self.fail_match_dict['image_path'])
        
        # 最後跳出成功的張數，還有比對失敗的樟樹
        messagebox.showinfo('比對統計','比對統計\n成功張數: {}張, 失敗張數: {}張'.format(sucs_count, fail_count))
        
        self.success_label.pack_forget()
        self.success_lb.pack_forget()
        self.fail_label.pack_forget()
        self.fail_lb.pack_forget()
        
        self.export_json_btn.pack(fill=tk.X)
        self.success_label.pack(fill=tk.BOTH)
        self.success_lb.pack(fill=tk.BOTH, expand=1)
        self.fail_label.pack(fill=tk.BOTH)
        self.fail_lb.pack(fill=tk.BOTH, expand=1)
        
    def fail_select_event(self, event):
        
        if not self.fail_lb.curselection():
            return
        
        print('fail_select_event!!!')
        select_index=self.fail_lb.curselection()[0]
        
        select_graphic = self.fail_match_dict['graphic'][select_index]
        
        select_graphic = cv2Tk_image(select_graphic)
        self.show_image_label.imgtk = select_graphic
        self.show_image_label.configure(image=select_graphic)
        
        # 顯示出圖片的Label
        self.show_image_label.pack()
        
    def success_select_event(self, event):
        if not self.success_lb.curselection():
            return
        print('success_select_event!!!')
        select_index = self.success_lb.curselection()[0]

        select_graphic = self.success_match_dict['graphic'][select_index]
        select_graphic = cv2Tk_image(select_graphic)
        self.show_image_label.imgtk = select_graphic
        self.show_image_label.configure(image=select_graphic)

        # 顯示出圖片的Label
        self.show_image_label.pack()
        
    def export_json(self,):
        save_name = filedialog.asksaveasfilename(defaultextension=".json")
        if save_name:
            data = {'success': {'image_path': self.success_match_dict['image_path'],
                                'face_names': self.success_match_dict['face_names'],
                                'locations': self.success_match_dict['locations']}, 
                    'fail': {'image_path': self.fail_match_dict['image_path'],
                                'face_names': self.fail_match_dict['face_names'],
                                'locations': self.fail_match_dict['locations']}}
        
            # 將訓練好的資料儲存成pickle格式的檔案，檔名=file_name
            with open(save_name, 'w') as f:
                json.dump(data, f, sort_keys=True)
            print('帥！成功export一個JSON資料')
        
        
        
    def __init__(self, master, train_data):
        self.root = tk.Toplevel()
        self.root.title('相片分析')
        self.root.geometry('1200x800')
        
        # 取得目前鍛鍊的訓練資料
        self.train_data = train_data
        
        # -------------------------------
        #    正上方的圖片輸入＆人臉辨識的按鈕
        # -------------------------------
        self.frame_top = tk.Frame(self.root)
        self.frame_top.pack(side=tk.TOP, fill=tk.X)
        
        # 顯示提示文字的Label
        self.message_label = tk.Label(self.frame_top, bg='yellow', width=20, text='empty')
        self.message_label.pack()
        
        # 設定讀取圖片的按鈕
        self.read_file_btn = tk.Button(self.frame_top, text='讀取圖片', command=self.read_image)
        self.read_file_btn.pack()
        
        # 點選變會開始分析圖片 的按鈕(預設影藏)
        self.analysis_faces_btn = tk.Button(self.frame_top, text='人臉辨識', command=self.face_detect_show_image)
        
        
        # 顯示結果圖片的Label(預設影藏)
        self.show_image_label = tk.Label(self.root, )
        
                        
        # -------------------------------
        #       左邊的列表＆ 輸出按鈕
        # -------------------------------
        self.frame_left = tk.Frame(self.root)
        self.frame_left.pack(side=tk.LEFT, fill=tk.Y)
        
        # 預設隱藏
        self.export_json_btn = tk.Button(self.frame_left, text='輸出JSON格式', command=self.export_json)
        
        # <視窗元件> 標籤(成功)
        self.success_label = tk.Label(self.frame_left, text='比對成功')
        
        # <視窗元件> 比對成功的列表
        self.success_lb = tk.Listbox(self.frame_left)
        
        # <視窗元件> 標籤(失敗)
        self.fail_label = tk.Label(self.frame_left, text='比對失敗')
        
        # <視窗元件> 比對失敗的列表
        self.fail_lb = tk.Listbox(self.frame_left)
#         for data in self.train_data['face_names']:
#             self.success_lb.insert('end', data)

        # <視窗元件> 排版在左邊
        self.success_label.pack(fill=tk.BOTH)
        self.success_lb.pack(fill=tk.BOTH, expand=1)
        self.fail_label.pack(fill=tk.BOTH)
        self.fail_lb.pack(fill=tk.BOTH, expand=1)
        
        # <事件函數> 當列表被選取時，會觸發self.listbox_select_event函數
        self.fail_lb.bind('<<ListboxSelect>>', self.fail_select_event)
        self.success_lb.bind('<<ListboxSelect>>', self.success_select_event)
        
        
        
    def quit(self):
        pass
    
# 這邊只是測試用。
file_name = filedialog.askopenfilename(filetypes=(('PICKLE', ".pickle"), ('trainData', '.pktr')))
my_train_data = dict()

print('file_name=', file_name)
if file_name:
    # 讀取pickle格式的檔案加入train_data，檔名=file_name
    with open(file_name, 'rb') as f:
        import_data = pickle.load(f)
        my_train_data['face_draws'] = import_data['face_draws']
        my_train_data['face_names'] = import_data['face_names']
        my_train_data['face_encodings'] = import_data['face_encodings']
    print('帥！成功import')
    testRoot = tk.Tk()
    test_imageFrame = ImageAnalysisFrame(master=testRoot, train_data=my_train_data)
    testRoot.mainloop()
else:
    print('哭！無法import')

file_name= /Users/guzongjia/OpenCV-Python/face_tracking/test.pktr
帥！成功import
file_path : /Users/guzongjia/OpenCV-Python/face_tracking/people6.JPG
file_path : /Users/guzongjia/OpenCV-Python/face_tracking/people5.JPG
file_path : /Users/guzongjia/OpenCV-Python/face_tracking/people4.JPG
file_path : /Users/guzongjia/OpenCV-Python/face_tracking/people3.jpg
file_path : /Users/guzongjia/OpenCV-Python/face_tracking/people3.png
file_path : /Users/guzongjia/OpenCV-Python/face_tracking/people1.jpg
match: [False, False, False, False, False, False, False, False]
max(face_names_times_dict) =  obama
match: [False, False, False, False, False, False, False, False]
max(face_names_times_dict) =  obama
match: [False, False, False, False, False, False, False, False]
max(face_names_times_dict) =  obama
match: [False, False, False, False, False, False, False, False]
max(face_names_times_dict) =  obama
match: [False, False, False, False, False, False, False, False]
max(face_names_times_dict) =  obama
match: [F

match: [False, False, False, False, False, False, False, False]
max(face_names_times_dict) =  obama
match: [True, True, True, True, True, False, False, False]
max(face_names_times_dict) =  obama
match: [False, False, False, False, False, True, False, False]
max(face_names_times_dict) =  obama
match: [False, False, False, False, False, False, True, True]
max(face_names_times_dict) =  obama
match: [False, False, False, False, False, False, False, False]
max(face_names_times_dict) =  obama
fail_select_event!!!
fail_select_event!!!
fail_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_select_event!!!
success_selec

In [3]:
# 相機即時分析模式
class CamereaAnalysisFrame:
    cancel_ok = None
    face_names, face_locations = [], []
    
    def __init__(self, master, train_data):
        self.root = tk.Toplevel()
        self.train_data = train_data
        self.root.geometry('1200x800')
        
        # 人臉名稱的集合
        self.face_set = set(train_data['face_names'])
        
        # 創建一個tracker字典，這邊只是
        self.trackers_dict = dict()
        for face_name in self.face_set:
            self.trackers_dict[face_name] = dlib.correlation_tracker()
            
        # tracking_face[人名] 等於True 代表此人名的臉正被追蹤; 反之，False代表未追蹤。
        self.tracking_faces = dict()
        for face_name in self.face_set:
            self.tracking_faces[face_name] = False
            
        # 決定是否要分析當前Frame的布林變數
        self.process_this_frame = True
        
        # 加速frame分析的開關(但快就代表會下降精確度)
        self.speed_up_correct_down = True
        
        # <視窗元件> 啟動相機按鈕
        self.camera_start_btn = tk.Button(self.root, text='啟動相機(開始分析)',command=self.camera_start)
        self.camera_start_btn.pack()
        
        # <視窗元件> 返回按鈕
        self.back_btn = tk.Button(self.root, text='返回',command=self.quit)
        self.back_btn.pack()
        

        
        # <視窗元件> 顯示相機分析畫面的 Label
        self.camera_frame_label = tk.Label(self.root)
        self.camera_frame_label.pack()
        
    def camera_read_frame(self):
        global cancel_ok
        
        print('相機讀取')
        if self.camera_open:
            print('相機讀取')
            _, self.frame = self.cap.read()
#             self.graphic = self.frame.copy()
            self.graphic = self.frame
            
            if self.speed_up_correct_down:
                small_frame = cv2.resize(self.frame, (0, 0), fx=0.5, fy=0.5)
            else:
                small_frame = self.frame
    
#----top----top----top----top----top----top----top----top----top----top----top----top----top-----
        # 追蹤區塊開頭----追蹤中---------------------------------------------------------                        
            # 使用dlib.tracker時，將frame複製一份給graphic
#             if any(self.tracking_faces.values()):
#                 self.graphic = self.frame.copy()                
            
            # 使用dlib.tracker時，不在乎是否要跳Frame處理    
            for face_name in self.tracking_faces:
                if self.tracking_faces[face_name]:
            #         print('tracking中')
                    # Update the tracker and request information about the
                    # quality of the tracking update
                    trackingQuality = self.trackers_dict[face_name].update(small_frame)

                    # If the tracking quality is good enough, determine the
                    # updated position of the tracked region and draw the
                    # rectangle
                    # trackingQuality越高，代表越嚴格的追蹤(太不像就會放棄追蹤)
                    if trackingQuality >= 4.75:
            #             print('tracking到好目標，繼續tracking')
                        tracked_position = self.trackers_dict[face_name].get_position()

                        left = int(tracked_position.left())
                        top = int(tracked_position.top())
                        right = left + int(tracked_position.width())
                        bottom = top + int(tracked_position.height())
                        if self.speed_up_correct_down:
                            left *= 2
                            top *= 2
                            right *= 2
                            bottom *= 2
                        
                        cv2.rectangle(self.graphic, (left, top),
                                        (right, bottom),
                                        RECTANGLE_COLOR ,2)
                        font = cv2.FONT_HERSHEY_DUPLEX
                        cv2.putText(self.graphic, face_name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
                    else:
                        # 放棄追蹤，結束tracking
            #             print('tracking不到好目標，設定tarckingFace=0')
                        self.tracking_faces[face_name] = False
        # 追蹤區塊底部----追蹤中---------------------------------------------------
# ---bottom----bottom----bottom----bottom----bottom----bottom----bottom----bottom----bottom----

            # 進行比對人臉分析
            if self.process_this_frame:
#                 small_frame = cv2.resize(self.frame, (0, 0), fx=0.5, fy=0.5)
#                 small_frame = self.frame
                self.face_locations = face_recognition.face_locations(small_frame)

                face_encodings = face_recognition.face_encodings(small_frame, self.face_locations)

                # 比對到的人名
                self.face_names = []
                known_encodings = self.train_data['face_encodings']

                # 已知的人名集合(不重複)
                face_names_set = set(self.train_data['face_names'])

                # 已知的人名對應出現次數的字典
                face_names_times_dict = {}

                # 找出所有臉中是否有比對到臉的，有的話就添加(投票機制)
                for face_encoding in face_encodings:
                    match = face_recognition.compare_faces(known_encodings , face_encoding, COMPARE_THRESHOLD)
                    name = "Unknown"
                    if len(known_encodings) > 0:
                        print('match:', match)

                        # 初始化 人名對次數的字典
                        for name in face_names_set:
                            face_names_times_dict[name] = 0

                        # 第一個比對成功到的臉的名字，不管之後有沒有類似的(投票機制)
                        for inx, value in enumerate(match):
                            if value:
                                name = self.train_data['face_names'][inx]
                                face_names_times_dict[name] += 1

                        # 如果字典最大值等於0，代表說沒有一張臉比對成功
                        print('max(face_names_times_dict)=', max(face_names_times_dict))
                        if [value for (inx, value) in face_names_times_dict.items() if value == max(face_names_times_dict.values())][0] <= 0:
                            self.face_names.append('Unkown')
                            continue

                        # 找出字典中值最大的key（此邊的Key為 樣本名稱）
                        print('max: '+max(face_names_times_dict, key=face_names_times_dict.get))
                        self.face_names.append(max(face_names_times_dict, key=face_names_times_dict.get))

                    else: # train_data什麼資料都沒有，直接判定unknow
                        self.face_names.append(name)
                        
#----top----top----top----top----top----top----top----top----top----top----top----top----top-----
        # 追蹤區塊頂部----啟動追蹤---------------------------------------------------
                # 如果有比對到在self開始追蹤
                for FACE_NAME in self.face_set:
                    if FACE_NAME in self.face_names:
                        face_index = self.face_names.index(FACE_NAME)

                        # face, and we determine this based on the largest
                        # area of the found rectangle. First initialize the
                        # required variables to 0
                        maxArea = 0
                        x = 0
                        y = 0
                        w = 0
                        h = 0
                        # Loop over all faces and check if the area for this
                        # face is the largest so far
                        # We need to convert it to int here because of the
                        # requirement of the dlib tracker. If we omit the cast to
                        # int here, you will get cast errors since the detector
                        # returns numpy.int32 and the tracker requires an int
                        (_top, _right, _bottom, _left) = self.face_locations[face_index]

                        if (_bottom - _top) * (_right - _left) > maxArea:
                            maxArea = (_bottom - _top) * (_right - _left)
        #                     print('maxArea:', maxArea)
                        # If one or more faces are found, initialize the tracker
                        # on the largest face in the picture
                        if maxArea > 0:
                            # Initialize the tracker
                            self.trackers_dict[FACE_NAME].start_track(small_frame,
                                                dlib.rectangle(_left - 10,
                                                               _top - 20,
                                                               _right + 10,
                                                               _bottom+ 20))

                            # Set the indicator variable such that we know the
                            # tracker is tracking a region in the image
                            self.tracking_faces[FACE_NAME] = True

  
        # 追蹤區塊底部----啟動追蹤---------------------------------------------------
# ---bottom----bottom----bottom----bottom----bottom----bottom----bottom----bottom----bottom----

            # 參考最新的位置.人名進行畫圖
            for inx, (top, right, bottom, left) in enumerate(self.face_locations):
                if self.speed_up_correct_down:
                    top *= 2
                    right *= 2
                    bottom *= 2
                    left *= 2
                
                cv2.rectangle(self.graphic, (left, top), (right, bottom), (0, 0, 255), 2)

                # Draw a label with a name below the face
                cv2.rectangle(self.frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(self.graphic, str(inx)+self.face_names[inx], (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            
            # 這次有分析，則下次不分析; 反之亦然
            print('process_this_frame:', self.process_this_frame)
            self.process_this_frame = not self.process_this_frame

            capture_image = cv2Tk_image(self.graphic)
            self.camera_frame_label.imgtk = capture_image
            self.camera_frame_label.configure(image=capture_image)
            cancel_ok = self.camera_frame_label.after(20, self.camera_read_frame)            
            
    def camera_stop(self):
        global cancel_ok
        cancel_ok = False
        print('self.camera_open = ', self.camera_open)
        self.camera_frame_label.after_cancel(self.cancel_id)
        
        self.camera_frame_label.pack_forget()
        self.back_btn.pack()
        
        # 停止錄影(關閉相機)
        self.cap.release()
        
        # 按下關閉相機按鈕，將文字改成 '啟動相機'
        self.camera_start_btn.configure(text='啟動相機(開始分析)')
        self.camera_start_btn.configure(command=self.camera_start)
        
    def camera_start(self):
        # 按下啟動相機按鈕，將文字改成 '關閉相機'
        self.camera_start_btn.configure(text='關閉相機(停止分析)',command=self.camera_stop)
        self.back_btn.pack_forget()
        self.camera_frame_label.pack()
#         self.camera_start_btn.configure(command=self.camera_stop)

        # 開始錄影(啟動相機)
        self.cap = cv2.VideoCapture(0)
        width, height = 1000, 800
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        
        # 設定錄影的暫停變數!
        self.capture_one = False
        self.camera_open = True
        self.cancel_id = 0
        print('camera_start(self)')
        self.camera_read_frame()

    def quit(self):
        self.root.destroy()
        self.cap.release()
    

In [4]:

class TrainFrame:
    '''
    訓練模式的視窗
    功能：
        負責使用相機或新增圖檔的方式，添加可比對的人臉
    '''
    
    '''按下"圖像輸入"，
    會新增一張臉的三種資訊到train_data中'''
    def not_found_show_image(self):
        cvImage = cv2.imread('eye.jpeg')
        cvImage = cv2.resize(cvImage, (900, 900))
        capture_image = cv2Tk_image(cvImage)
        self.show_img_label.imgtk = capture_image
        self.show_img_label.configure(image=capture_image)
    def listbox_select_event(self, event):
        if not self.faces_lb.curselection():
            return
        select_index=self.faces_lb.curselection()[0]
        value = self.train_data['face_names'][select_index]
        
        select_graphic = self.train_data['face_draws'][select_index]
        select_graphic = cv2Tk_image(select_graphic)
        self.show_img_label.imgtk = select_graphic
        self.show_img_label.configure(image=select_graphic)

        print('value:', value)
        self.face_name_variable.set(value)
        print('event觸發囉！')
        
    def camera_read_frame(self):
        if self.camera_open:
            print('相機讀取')
            _, self.frame = self.cap.read()
            
            capture_image = cv2Tk_image(self.frame)
            self.show_img_label.imgtk = capture_image
            self.show_img_label.configure(image=capture_image)
            self.cancel_id = self.show_img_label.after(50, self.camera_read_frame)            
            
    def camera_close(self):
        self.camera_open = False
        print('關閉中...')
        self.show_img_label.after_cancel(self.cancel_id)
        print('相機關閉')
        self.cap.release()
        print('重新生成 相機輸入按鈕，隱藏關閉按鈕')
        self.close_btn.pack_forget()
        self.face_image_input_btn.pack(side=tk.LEFT, padx=10)
        self.face_camera_input_btn.pack(side=tk.LEFT, padx=10)
        
        # 暫停相機後，會將當前的圖進行人臉辨識！！
        face_locations = face_recognition.face_locations(self.frame)
        
        # 圖片沒有找到人臉，失敗return
        if len(face_locations) < 1:
            self.not_found_show_image()
            messagebox.showinfo('辨識結果', '並未找到人臉在該圖片上')
            return
        
        
        face_encodings = face_recognition.face_encodings(self.frame, face_locations)[0]
        graphic = self.frame
            
        # 將捕捉到的臉用藍色框框畫在圖片上
        top, right, bottom, left = face_locations[0]
        cv2.rectangle(graphic, (left, top), (right, bottom), (0, 0, 255), 2)
        
        # 最後將圖片顯示在正中央區塊的Label上
        capture_image = cv2Tk_image(graphic)
        self.show_img_label.imgtk = capture_image
        self.show_img_label.configure(image=capture_image)
        
        # 使用者輸入的新樣本名稱
        face_name = self.face_name_entry.get() # 使用者輸入的人臉名稱
        
        # 最後跳出訊息視窗，詢問是否要儲存成樣本
        user_answer = messagebox.askokcancel('樣本存檔', '是否要儲存成'+self.face_name_entry.get()+'的樣本?')
        print('user_answer=', user_answer)
        
        # 使用者回答正確會回傳True
        if user_answer:
            self.faces_add(face_encodings, face_name, graphic)
        else:# 不存成，就會覆蓋掉相片畫面並且也不添加此項到train_data中
            self.not_found_show_image()
            
    def face_add_by_camera_input(self,):            
        if self.face_name_entry.get() == '':
            print('樣本名稱不得為空')
            print(messagebox.showinfo('警告', '樣本名稱不得為空'))
            return
        
        self.face_camera_input_btn.pack_forget()
        self.face_image_input_btn.pack_forget()
        self.close_btn.pack(side=tk.LEFT, padx=10)
        
        # 開始錄影
        self.cap = cv2.VideoCapture(0)
        width, height = 1066, 600
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        
        # 設定錄影的暫停變數!
        self.capture_one = False
        self.camera_open = True
        self.cancel_id = 0
        self.camera_read_frame()
        pass
    
    def face_add_by_image_input(self,):
        if self.face_name_entry.get() == '':
            print('樣本名稱不得為空')
            print(messagebox.showinfo('警告', '樣本名稱不得為空'))
            return
        # 開啟讀取圖片檔案的視窗
        image_file_names = filedialog.askopenfilenames(filetypes=(('JPG', ".jpg"), ('JPEG', ".jpeg"), ('PNG', '.png')))
        
        count_faces = 0
        
        # 圖片input沒成功載入(可能圖片路徑失效or副檔名不對or關閉讀檔視窗)
        if len(image_file_names) < 1:
            self.not_found_show_image()
            print('並未載入圖片')
            return 
        for image_file_name in image_file_names:
            # 圖片路徑讀取成功，利用cv2讀圖片檔
            cvImg = cv2.imread(image_file_name)

            face_locations = face_recognition.face_locations(cvImg)

            # 圖片沒有找到人臉，失敗return
            if len(face_locations) < 1:
                self.not_found_show_image()
                continue
                
            # 有在此圖找到一張臉
            count_faces += 1
            
            face_encodings = face_recognition.face_encodings(cvImg, face_locations)[0]
            graphic = cvImg.copy()

            # 將捕捉到的臉用藍色框框畫在圖片上
            top, right, bottom, left = face_locations[0]
            cv2.rectangle(graphic, (left, top), (right, bottom), (0, 0, 255), 2)

            # 使用者輸入的新樣本名稱
            face_name = self.face_name_entry.get() # 使用者輸入的人臉名稱

            # 添加合格的人臉資料(比對圖.人臉名稱.展示圖)到train_data之中
            self.faces_add(face_encodings, face_name, graphic)
        
        count_not_found_faces = len(image_file_names) - count_faces
        
        messagebox.showinfo('辨識結果', '共有{}張臉辨識成功，{}張圖辨識失敗'.format(count_faces, count_not_found_faces))

    def faces_add(self, face_encodings, face_name, graphic):
        if face_name in self.train_data['face_names']:
            insert_inx = self.train_data['face_names'].index(face_name)
            self.train_data['face_draws'].insert(insert_inx, graphic)
            self.train_data['face_names'].insert(insert_inx, face_name)
            self.train_data['face_encodings'].insert(insert_inx, face_encodings)
            
            # 視窗上的樣本列表，插入一個名字
            self.faces_lb.insert(insert_inx, face_name)
        else:
            print('train_data還沒有跟此相同名稱的臉')
            self.train_data['face_draws'].append(graphic)
            self.train_data['face_names'].append(self.face_name_entry.get())
            self.train_data['face_encodings'].append(face_encodings)
            # 視窗上的樣本列表，在最後面添加一個名字
            self.faces_lb.insert('end', face_name)
        
    
    def faces_delete(self,):
        print('請先選取要刪除的樣本，進行刪除！')
        if self.faces_lb.curselection():
            delete_inx = self.faces_lb.curselection()[0]
            del self.train_data['face_draws'][delete_inx]
            del self.train_data['face_names'][delete_inx]
            del self.train_data['face_encodings'][delete_inx]
            self.faces_lb.delete(delete_inx)
        else:
            print('不行不行！')
    
    def __init__(self, master=None, train_data=None):
        # 如果沒有上層的tk視窗參數，就把此物件做為主視窗
        if master is None:
            self.root = tk.Tk()
        else:
            self.root = tk.Toplevel(master)
            self.master = master
            
        if train_data is None:
            train_data = {'face_draws': [],
                              'face_names': [],
                              'face_encodings': []}
            print('!!', train_data)
        self.train_data = train_data
        
        self.root.title('訓練模式')
        self.root.geometry('1200x800')
        
        # ------------------------------
        # ------- 視窗元件的排版    ------
        # ------------------------------
        self.root.configure(background='gray')
        
# -------------------------------
#       左邊的列表＆刪除按鈕
# -------------------------------
        self.frame_left = tk.Frame(self.root)
        self.frame_left.pack(side=tk.LEFT, fill=tk.Y)
        
        # <視窗元件> 名稱列表的刪除按鈕
        self.faces_delete_btn = tk.Button(self.frame_left, text='刪除選取的樣本', command=self.faces_delete)
        
        # <視窗元件> 顯示目前已訓練臉的名稱列表
        self.faces_lb = tk.Listbox(self.frame_left)
        
        for data in self.train_data['face_names']:
            self.faces_lb.insert('end', data)

        self.faces_delete_btn.pack(fill=tk.BOTH)
        self.faces_lb.pack(fill=tk.BOTH, expand=1)
        
        # <事件函數> 當列表被選取時，會觸發self.listbox_select_event函數
        self.faces_lb.bind('<<ListboxSelect>>', self.listbox_select_event)
# -------------------------------
#       右上部分的功能選單
# -------------------------------
        self.frame_rtop = tk.Frame(self.root, background='gray')
        self.frame_rtop.pack(side=tk.TOP, fill=tk.X)
        # 
        self.face_name_label = tk.Label(self.frame_rtop, text='新樣本名稱', bg='yellow',)
        
        # <視窗元件> 輸入人臉名稱entry
        self.face_name_variable = tk.StringVar(self.root) 
        self.face_name_entry = tk.Entry(self.frame_rtop, bg='yellow',textvariable=self.face_name_variable)
       
        
        # <視窗元件> 圖檔輸入按鈕
        self.face_image_input_btn = tk.Button(self.frame_rtop, 
                                              text='多圖檔輸入', 
                                             command=self.face_add_by_image_input)
        # <視窗元件> 相機拍照輸入按鈕
        self.face_camera_input_btn = tk.Button(self.frame_rtop, 
                                              text='相機輸入', 
                                             command=self.face_add_by_camera_input)
        self.camera_open = False
        
        # <視窗元件> 相機暫停按鈕
        self.close_btn = tk.Button(self.frame_rtop, text='點一下關閉', command=self.camera_close)
        
        self.face_name_label.pack(side=tk.LEFT, padx=10)
        self.face_name_entry.pack(side=tk.LEFT, padx=10)
        self.face_image_input_btn.pack(side=tk.LEFT, padx=10)
        self.face_camera_input_btn.pack(side=tk.LEFT, padx=10)
        self.close_btn.pack(side=tk.LEFT, padx=10)
        self.close_btn.pack_forget()

# -------------------------------
#       右半邊的 圖片展示區
# -------------------------------
        self.frame_rcenter = tk.Frame(self.root, background='green')
        self.frame_rcenter.pack(fill=tk.BOTH)
        
        # <視窗元件> 圖片展示區
        self.show_img_label = tk.Label(self.frame_rcenter, background='green')
        
        self.not_found_show_image()
        
        self.show_img_label.pack()
# -----
        
        print('yoooo')
        
        if master is None:
            self.root.mainloop()

# testRoot = tk.Tk()
# test_trainFrame = TrainFrame(master=testRoot)
# testRoot.mainloop()

print('ok')

ok


In [5]:
# 影片分析模式
class VideoAnalysisFrame:
    cancel_ok = None
    face_names, face_locations = [], []
    
    def __init__(self, master, train_data):
        self.root = tk.Toplevel()
        self.train_data = train_data
        self.root.title('影片分析模式')
        self.root.geometry('1200x800')
        
        # 人臉名稱的集合
        self.face_set = set(train_data['face_names'])
        
        # 創建一個tracker字典，這邊只是
        self.trackers_dict = dict()
        for face_name in self.face_set:
            self.trackers_dict[face_name] = dlib.correlation_tracker()
            
        # tracking_face[人名] 等於True 代表此人名的臉正被追蹤; 反之，False代表未追蹤。
        self.tracking_faces = dict()
        for face_name in self.face_set:
            self.tracking_faces[face_name] = False
        
        # 加速frame分析的開關(但快就代表會下降精確度)
        self.speed_up_correct_down = True
        
        # 決定是否要分析當前Frame的布林變數
        self.process_this_frame = True
        
        # <視窗元件> 啟動相機按鈕
        self.camera_start_btn = tk.Button(self.root, text='啟動相機(開始分析)',command=self.camera_start)
        self.camera_start_btn.pack()
        
        # <視窗元件> 返回按鈕
        self.back_btn = tk.Button(self.root, text='返回',command=self.quit)
        self.back_btn.pack()
        
        # <視窗元件> 顯示相機分析畫面的 Label
        self.camera_frame_label = tk.Label(self.root)
        self.camera_frame_label.pack()
        
    def camera_read_frame(self):
        global cancel_ok
        
        print('相機讀取')
        if self.camera_open:
            print('相機讀取')
            # 當isOpened() == False時，執行底下關閉相機&恢復原本樣貌
            if not self.cap.isOpened():
                self.camera_stop()
                return
    
            _, self.frame = self.cap.read()
#             self.graphic = self.frame.copy()
            self.graphic = self.frame
            
            # 判斷是否要辨識加速
            if self.speed_up_correct_down:
                small_frame = cv2.resize(self.frame, (0, 0), fx=0.5, fy=0.5)
            else:
                small_frame = self.frame
            
#----top----top----top----top----top----top----top----top----top----top----top----top----top-----
        # 追蹤區塊開頭----追蹤中---------------------------------------------------------                        
            # 使用dlib.tracker時，將frame複製一份給graphic
#             if any(self.tracking_faces.values()):
#                 self.graphic = self.frame.copy()                
            
            # 使用dlib.tracker時，不在乎是否要跳Frame處理    
            for face_name in self.tracking_faces:
                if self.tracking_faces[face_name]:
            #         print('tracking中')
                    # Update the tracker and request information about the
                    # quality of the tracking update
                    trackingQuality = self.trackers_dict[face_name].update(small_frame)

                    # If the tracking quality is good enough, determine the
                    # updated position of the tracked region and draw the
                    # rectangle
                    # trackingQuality越高，代表越嚴格的追蹤(太不像就會放棄追蹤)
                    if trackingQuality >= 4.75:
            #             print('tracking到好目標，繼續tracking')
                        tracked_position = self.trackers_dict[face_name].get_position()

                        left = int(tracked_position.left())
                        top = int(tracked_position.top())
                        right = left + int(tracked_position.width())
                        bottom = top + int(tracked_position.height())
                        
                        if self.speed_up_correct_down:
                            left *= 2
                            top *= 2
                            right *= 2
                            bottom *= 2
                            
                        cv2.rectangle(self.graphic, (left, top),
                                        (right, bottom),
                                        RECTANGLE_COLOR ,2)
                        font = cv2.FONT_HERSHEY_DUPLEX
                        cv2.putText(self.graphic, face_name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
                    else:
                        # 放棄追蹤，結束tracking
            #             print('tracking不到好目標，設定tarckingFace=0')
                        self.tracking_faces[face_name] = False
        # 追蹤區塊底部----追蹤中---------------------------------------------------
# ---bottom----bottom----bottom----bottom----bottom----bottom----bottom----bottom----bottom----

            # 進行比對人臉分析
            if self.process_this_frame:
                self.face_locations = face_recognition.face_locations(small_frame)

                face_encodings = face_recognition.face_encodings(small_frame, self.face_locations)

                # 比對到的人名
                self.face_names = []
                known_encodings = self.train_data['face_encodings']

                # 已知的人名集合(不重複)
                face_names_set = set(self.train_data['face_names'])

                # 已知的人名對應出現次數的字典
                face_names_times_dict = {}

                # 找出所有臉中是否有比對到臉的，有的話就添加(投票機制)
                for face_encoding in face_encodings:
                    match = face_recognition.compare_faces(known_encodings , face_encoding, COMPARE_THRESHOLD)
                    name = "Unknown"
                    if len(known_encodings) > 0:
                        print('match:', match)

                        # 初始化 人名對次數的字典
                        for name in face_names_set:
                            face_names_times_dict[name] = 0

                        # 第一個比對成功到的臉的名字，不管之後有沒有類似的(投票機制)
                        for inx, value in enumerate(match):
                            if value:
                                name = self.train_data['face_names'][inx]
                                face_names_times_dict[name] += 1

                        # 如果字典最大值等於0，代表說沒有一張臉比對成功
                        print('max(face_names_times_dict)=', max(face_names_times_dict))
                        if [value for (inx, value) in face_names_times_dict.items() if value == max(face_names_times_dict.values())][0] <= 0:
                            self.face_names.append('Unkown')
                            continue

                        # 找出字典中值最大的key（此邊的Key為 樣本名稱）
                        print('max: '+max(face_names_times_dict, key=face_names_times_dict.get))
                        self.face_names.append(max(face_names_times_dict, key=face_names_times_dict.get))

                    else: # train_data什麼資料都沒有，直接判定unknow
                        self.face_names.append(name)
                        
#----top----top----top----top----top----top----top----top----top----top----top----top----top-----
        # 追蹤區塊頂部----啟動追蹤---------------------------------------------------
                # 如果有比對到在self開始追蹤
                for FACE_NAME in self.face_set:
                    if FACE_NAME in self.face_names:
                        face_index = self.face_names.index(FACE_NAME)

                        # face, and we determine this based on the largest
                        # area of the found rectangle. First initialize the
                        # required variables to 0
                        maxArea = 0
                        x = 0
                        y = 0
                        w = 0
                        h = 0
                        # Loop over all faces and check if the area for this
                        # face is the largest so far
                        # We need to convert it to int here because of the
                        # requirement of the dlib tracker. If we omit the cast to
                        # int here, you will get cast errors since the detector
                        # returns numpy.int32 and the tracker requires an int
                        (_top, _right, _bottom, _left) = self.face_locations[face_index]
                            
                        if (_bottom - _top) * (_right - _left) > maxArea:
                            maxArea = (_bottom - _top) * (_right - _left)
        #                     print('maxArea:', maxArea)
                        # If one or more faces are found, initialize the tracker
                        # on the largest face in the picture
                        if maxArea > 0:
                            # Initialize the tracker
                            self.trackers_dict[FACE_NAME].start_track(self.frame,
                                                dlib.rectangle(_left - 10,
                                                               _top - 20,
                                                               _right + 10,
                                                               _bottom+ 20))

                            # Set the indicator variable such that we know the
                            # tracker is tracking a region in the image
                            self.tracking_faces[FACE_NAME] = True

  
        # 追蹤區塊底部----啟動追蹤---------------------------------------------------
# ---bottom----bottom----bottom----bottom----bottom----bottom----bottom----bottom----bottom----

            # 參考最新的位置.人名進行畫圖
            for inx, (top, right, bottom, left) in enumerate(self.face_locations):
                if self.speed_up_correct_down:
                    top *= 2
                    right *= 2
                    bottom *= 2
                    left *= 2
                
                cv2.rectangle(self.graphic, (left, top), (right, bottom), (0, 0, 255), 2)

                # Draw a label with a name below the face
                cv2.rectangle(self.graphic, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(self.graphic, str(inx)+self.face_names[inx], (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            
            # 這次有分析，則下次不分析; 反之亦然
            print('process_this_frame:', self.process_this_frame)
            self.process_this_frame = not self.process_this_frame

            capture_image = cv2Tk_image(self.graphic)
            self.camera_frame_label.imgtk = capture_image
            self.camera_frame_label.configure(image=capture_image)
            cancel_ok = self.camera_frame_label.after(20, self.camera_read_frame)            
            
    def camera_stop(self):
        global cancel_ok
        cancel_ok = False
        print('self.camera_open = ', self.camera_open)
        self.camera_frame_label.after_cancel(self.cancel_id)
        
        self.camera_frame_label.pack_forget()
        self.back_btn.pack()
        
        # 停止錄影(關閉相機)
        self.cap.release()
        
        # 按下關閉相機按鈕，將文字改成 '啟動相機'
        self.camera_start_btn.configure(text='啟動相機(開始分析)')
        self.camera_start_btn.configure(command=self.camera_start)
        
    def camera_start(self):
        # 跳出 讀取影片檔的視窗
        video_file_name = filedialog.askopenfile(filetypes=(('mp4', ".mp4"), ('avi', ".avi")))
        # 沒有成功輸入該檔案名稱，跳出
        print('video_file_name=', video_file_name)
        if not video_file_name:
            return
        
        # 按下啟動相機按鈕，將文字改成 '關閉相機'
        self.camera_start_btn.configure(text='關閉相機(停止分析)',command=self.camera_stop)
        self.back_btn.pack_forget()
        self.camera_frame_label.pack()
#         self.camera_start_btn.configure(command=self.camera_stop)

        # 開始錄影(啟動相機)
        self.cap = cv2.VideoCapture(video_file_name.name)
        width, height = 1000, 800
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        
        # 設定錄影的暫停變數!
        self.capture_one = False
        self.camera_open = True
        self.cancel_id = 0
        print('camera_start(self)')
        self.camera_read_frame()

    def quit(self):
        self.root.destroy()
        self.cap.release()


In [6]:

class App:
    def export_train_data(self,):
        file_name = filedialog.asksaveasfilename(defaultextension=".pktr")
        print('file_name=', file_name)
        if file_name:
            # 將訓練好的資料儲存成pickle格式的檔案，檔名=file_name
            with open(file_name, 'wb') as f:
                pickle.dump(self.train_data, f)
            print('帥！成功export')
            
    def import_train_data(self,):
        file_name = filedialog.askopenfilename(filetypes=(('PICKLE', ".pickle"), ('trainData', '.pktr')))

        print('file_name=', file_name)
        if file_name:
            # 讀取pickle格式的檔案加入train_data，檔名=file_name
            with open(file_name, 'rb') as f:
                import_data = pickle.load(f)
                self.train_data['face_draws'] = import_data['face_draws']
                self.train_data['face_names'] = import_data['face_names']
                self.train_data['face_encodings'] = import_data['face_encodings']
            print('帥！成功import')
    
    # 開啟視窗 <訓練模式>        
    def train_mode(self,):
        self.train_window = TrainFrame(master=self.root, train_data=self.train_data)
    
    # 開啟視窗 <相片分析模式>
    def image_mode(self):
        self.image_window = ImageAnalysisFrame(self.root, train_data=self.train_data)
    
    # 開啟視窗 <影片分析模式>
    def video_mode(self):
        self.video_window = VideoAnalysisFrame(self.root, train_data=self.train_data)
    
    # 開啟視窗 <相機即時分析模式>
    def camera_mode(self):
        # 相機即時分析
        self.camera_window = CamereaAnalysisFrame(self.root, train_data=self.train_data)  

        
        
    def input_selection(self):
        print('hello', self.var.get())
            
        if self.var.get() == '相片':
            print('相片')
            self.image_mode()
        elif self.var.get() == '影片':
            print('影片')
            self.video_mode()
            
        elif self.var.get() == '相機':
            print('相機')
            self.camera_mode()
            
    def debug_quit(self,):
        print('train_data', self.train_data)
        self.quit()
        
    def __init__(self):
        # 嘗試看看，能不能讓上層視窗共同操作一個List
        self.train_data = {'face_draws': [],
                      'face_names': [],
                      'face_encodings': []}
        self.root = tk.Tk()
        self.root.attributes('-topmost',True)
        self.root.overrideredirect(1)

        button = tk.Button(self.root, text = '離開系統', command=self.debug_quit)
        button.pack()
        
        self.root.title('人頭追蹤Plus!')
        self.root.geometry('200x200')
        
        self.tl = tk.Label(self.root, bg='green', width=20, text='訓練模式')
        self.tl.pack()
        
        self.train_btn = tk.Button(self.root, text='人臉搜集', command= self.train_mode )
        self.train_btn.pack()
        
        self.var = tk.StringVar(self.root)
        self.l = tk.Label(self.root, bg='yellow', width=20, text='分析模式')
        self.l.pack()


        self.r1 = tk.Radiobutton(self.root, text='相片分析',
            variable=self.var, value='相片',
            command=self.input_selection)
        self.r1.pack()
        self.r2 = tk.Radiobutton(self.root, text='影片分析',
            variable=self.var, value='影片',
            command=self.input_selection)
        self.r2.pack()

        self.r3 = tk.Radiobutton(self.root, text='相機即時分析',
            variable=self.var, value='相機',
            command=self.input_selection)
        self.r3.pack()
        
        # 新增menubar
        menubar = tk.Menu(self.root)
        filemenu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label='File', menu=filemenu)
        filemenu.add_command(label='export_train_data', command=self.export_train_data)
        filemenu.add_command(label='import_train_data', command=self.import_train_data)

        self.root.config(menu=menubar)
        
        self.root.mainloop()

    def quit(self):
        self.root.destroy()



In [7]:
def main():
    app = App()

if __name__ == '__main__':
    main()

hello 相片
相片
hello 影片
影片
video_file_name= <_io.TextIOWrapper name='/Users/guzongjia/OpenCV-Python/face_tracking/basic.mp4' mode='r' encoding='UTF-8'>
camera_start(self)
相機讀取
相機讀取
process_this_frame: True
相機讀取
相機讀取
process_this_frame: False
相機讀取
相機讀取
process_this_frame: True
相機讀取
相機讀取
process_this_frame: False
相機讀取
相機讀取
process_this_frame: True
相機讀取
相機讀取
process_this_frame: False
相機讀取
相機讀取
process_this_frame: True
相機讀取
相機讀取
process_this_frame: False
相機讀取
相機讀取
process_this_frame: True
相機讀取
相機讀取
process_this_frame: False
相機讀取
相機讀取
process_this_frame: True
相機讀取
相機讀取
process_this_frame: False
相機讀取
相機讀取
process_this_frame: True
相機讀取
相機讀取
process_this_frame: False
相機讀取
相機讀取
process_this_frame: True
相機讀取
相機讀取
process_this_frame: False
相機讀取
相機讀取
process_this_frame: True
相機讀取
相機讀取
process_this_frame: False
相機讀取
相機讀取
process_this_frame: True
self.camera_open =  True
相機讀取
相機讀取
self.camera_open =  True
hello 相機
相機


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/guzongjia/anaconda/envs/py35/lib/python3.5/tkinter/__init__.py", line 1558, in __call__
    return self.func(*args)
  File "<ipython-input-3-e8f9838e1959>", line 257, in quit
    self.cap.release()
AttributeError: 'CamereaAnalysisFrame' object has no attribute 'cap'


yoooo
請先選取要刪除的樣本，進行刪除！
不行不行！
train_data {'face_names': [], 'face_draws': [], 'face_encodings': []}


In [8]:
import tkinter as tk
from tkinter import ttk


class SampleApp(tk.Tk):

    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        self.button = ttk.Button(text="start", command=self.start)
        self.button.pack()
        self.progress = ttk.Progressbar(self, orient="horizontal",
                                        length=200, mode="determinate")
        self.progress.pack()

        self.bytes = 0
        self.maxbytes = 0

    def start(self):
        self.progress["value"] = 0
        self.maxbytes = 50000
        self.progress["maximum"] = 50000
        self.read_bytes()

    def read_bytes(self):
        '''simulate reading 500 bytes; update progress bar'''
        self.bytes += 500
        self.progress["value"] = self.bytes
        if self.bytes < self.maxbytes:
            # read more bytes after 100 ms
            self.after(100, self.read_bytes)

app = SampleApp()
app.mainloop()